In [59]:
import pyarrow.parquet as pq
import pyarrow as pa
import tqdm

# Specify the Parquet file paths
input_parquet_file_path = 'pairs_df.parquet'
output_parquet_file_path = 'pairs_df_with_rows.parquet'

# Define chunk size (number of rows to read and modify at a time)
chunk_size = 64_000  # Adjust this according to your needs

# Open the input Parquet file
input_parquet_file = pq.ParquetFile(input_parquet_file_path)

# Get the total number of rows
total_rows = input_parquet_file.metadata.num_rows

arrow_schema = input_parquet_file.schema.to_arrow_schema()

print(total_rows)

# delete pairs_df_with_rows.parquet if it already exists
import os
if os.path.exists(output_parquet_file_path):
    os.remove(output_parquet_file_path)


parquet_file = pq.ParquetFile(input_parquet_file_path)

output_writer = None

for batch in tqdm.tqdm(parquet_file.iter_batches(chunk_size), total=int(total_rows / chunk_size)):
    batch_df = batch.to_pandas()
    batch_df['row_id'] = batch_df['census_id'] + '_' + batch_df['deepdao_organization_id']

    modified_chunk_table = pa.Table.from_pandas(batch_df)

    # Append the modified chunk_table to the output Parquet file
    if not output_writer:
        # Create or open the output Parquet file
        output_writer = pq.ParquetWriter(
            output_parquet_file_path,
            modified_chunk_table.schema,
        )
    output_writer.write_table(modified_chunk_table)

# Close the output Parquet writer
output_writer.close()


66932677


  0%|          | 0/1045 [00:00<?, ?it/s]

1046it [00:47, 22.04it/s]                          


In [60]:
# Open the output Parquet file
output_parquet_file = pq.ParquetFile(output_parquet_file_path)

# Get the total number of rows
output_parquet_file.metadata.num_rows

66932677

In [61]:
output_parquet_file.schema

required group field_id=-1 schema {
  optional binary field_id=-1 census_id (String);
  optional binary field_id=-1 deepdao_organization_id (String);
  optional binary field_id=-1 deepdao_name (String);
  optional binary field_id=-1 census_name (String);
  optional binary field_id=-1 row_id (String);
}